<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/Depth_Anything_Reconstruction_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Depth Anything V1
Paper: https://arxiv.org/pdf/2401.10891.pdf<br>
Download a sample image

In [ ]:
!gdown 1L3mKl7qlUGE47wZydUST2aTXlBKpN-oI

Downloading...
From: https://drive.google.com/uc?id=1L3mKl7qlUGE47wZydUST2aTXlBKpN-oI
To: /content/img.png
100% 1.55M/1.55M [00:00<00:00, 76.7MB/s]


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip -q install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.5 MB/s eta 0:00:00


#Depth and Reconstruction: All Together

In [3]:
from transformers import AutoImageProcessor, AutoModelForDepthEstimation, DepthAnythingForDepthEstimation
import torch
from torchvision import transforms
import numpy as np
from PIL import Image
import requests
import matplotlib.pyplot as plt
from glob import glob
import matplotlib
import open3d as o3d
import cv2
import matplotlib as mpl
import matplotlib.cm as cm
import os

def render_depth_v2(values, colormap_name="magma_r") -> Image:
    vmax = np.percentile(values, 95)
    normalizer = mpl.colors.Normalize(vmin=values.min(), vmax=vmax)
    mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
    colormapped_im = (mapper.to_rgba(values)[:, :, :3] * 255).astype(np.uint8)
    return Image.fromarray(colormapped_im)

def reconstruct_pointcloud(rgb, depth, cam_K, vis_rgbd=False):

    rgb = np.asarray(rgb, order="C")
    rgb_im = o3d.geometry.Image(rgb.astype(np.uint8))
    depth_im = o3d.geometry.Image(depth)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(rgb_im, depth_im, convert_rgb_to_intensity=False)
    if vis_rgbd:
        plt.subplot(1, 2, 1)
        plt.title('RGB image')
        plt.imshow(rgbd_image.color)
        plt.subplot(1, 2, 2)
        plt.title('Depth image')
        plt.imshow(rgbd_image.depth)
        plt.colorbar()
        plt.show()

    cam = o3d.camera.PinholeCameraIntrinsic()
    cam.intrinsic_matrix = cam_K
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam)

    return pcd

def reconstruct_pointcloud_write(pred_depth, img_np, cam_K, ply_dir=None):

    scale = 1 #200
    MIN_DEPTH = 1e-3
    MAX_DEPTH = 400
    pred_depth *= scale
    pred_depth[pred_depth < MIN_DEPTH] = MIN_DEPTH
    pred_depth[pred_depth > MAX_DEPTH] = MAX_DEPTH
    print('Final depth min:',pred_depth.min(), ', max:', pred_depth.max())
    pcd = reconstruct_pointcloud(img_np, pred_depth, cam_K, vis_rgbd=False)
    o3d.io.write_point_cloud(ply_dir, pcd)
    print('3D Recon .ply saved in:', ply_dir)

def img_to_mask(img, reduction=40):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #print(np.unique(img_gray))
    mask = np.zeros(img_gray.shape)
    mask_final = np.zeros(mask.shape)
    mask[img_gray>0] = 255
    #print(np.unique(mask), mask.shape)
    mask = np.array(mask, np.uint8)
    circles = cv2.HoughCircles(mask, cv2.HOUGH_GRADIENT, 3.5, 100)
    if circles is not None:
        #print('found')
        circles = np.round(circles[0, :]).astype("int")
        (x, y, r) = circles[0:1][0]
        cv2.circle(mask_final, (x, y), r-reduction, 1, -1)
        img[mask_final==0] = 0
    else: print('not found:')
    return mask_final

def depth_prediction(img, image_processor, model):
    inputs = image_processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_depth = outputs.predicted_depth


    # print('raw input', img.size[::-1],'inputs',inputs['pixel_values'].shape, 'predicted_depth', predicted_depth.shape)
    prediction = predicted_depth.clone()

    #interpolate to original size
    prediction = torch.nn.functional.interpolate(
        predicted_depth.unsqueeze(1),
        size=img.size[::-1],
        mode="bicubic",
        align_corners=False,
    )

    output = prediction.squeeze().cpu().numpy()
    return output


device = 'cuda' if torch.cuda.is_available() else 'cpu'
fx = 850.709
# fx = 1050.709
img_width =  1280
img_height = 720
intrinsics = np.array([[fx / img_width, 0, 0.5, 0],
                [0, fx / img_height, 0.5, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])

intrinsics[0,:] *= img_width
intrinsics[1,:] *= img_height
cam_K = intrinsics[:3,:3]

# image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-small-hf")
# model = DepthAnythingForDepthEstimation.from_pretrained("LiheYoung/depth-anything-small-hf").to(device)
# model_weights = 'DA1_SMALL'

# image_processor = AutoImageProcessor.from_pretrained("LiheYoung/depth-anything-large-hf")
# model = DepthAnythingForDepthEstimation.from_pretrained("LiheYoung/depth-anything-large-hf").to(device)
# model_weights = 'DA1_LARGE'

# image_processor = AutoImageProcessor.from_pretrained("depth-anything/Depth-Anything-V2-Small-hf")
# model = DepthAnythingForDepthEstimation.from_pretrained("depth-anything/Depth-Anything-V2-Small-hf").to(device)
# model_weights = 'DA2_SMALL'

image_processor = AutoImageProcessor.from_pretrained("depth-anything/Depth-Anything-V2-Large-hf")
model = DepthAnythingForDepthEstimation.from_pretrained("depth-anything/Depth-Anything-V2-Large-hf").to(device)
model_weights = 'DA2_LARGE'

recon_dir = "/content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/{}/PLY".format(model_weights)
depth_dir = "/content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/{}/Depth_Pred".format(model_weights)
imgs_dirs = glob("/content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/imgs/*.png")
masked_imgs_dirs = "/content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/masked_imgs"
os.makedirs(recon_dir, mode = 0o777, exist_ok = True)
os.makedirs(depth_dir, mode = 0o777, exist_ok = True)
os.makedirs(masked_imgs_dirs, mode = 0o777, exist_ok = True)

for img_dir in imgs_dirs:
    img = Image.open(img_dir).convert("RGB")
    img_np = np.array(img)
    mask = img_to_mask(img_np, reduction=35)
    img_np[mask==0] = 0
    img = Image.fromarray(img_np)
    output = depth_prediction(img, image_processor, model)
    mask = img_to_mask(img_np, reduction=40)
    img_np[mask==0] = 0
    output[mask==0] = 0
    Image.fromarray(img_np).save(img_dir.replace('imgs', 'masked_imgs'))
    vis_pred_depth = render_depth_v2(output/255)
    file_name = os.path.basename(img_dir)
    vis_pred_depth.save(os.path.join(depth_dir, file_name))
    reconstruct_pointcloud_write(output, img_np, cam_K, ply_dir=os.path.join(recon_dir, file_name[:-3]+'ply'))


preprocessor_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Final depth min: 0.001 , max: 333.29272
3D Recon .ply saved in: /content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2_LARGE/PLY/0000097.ply
Final depth min: 0.001 , max: 342.09692
3D Recon .ply saved in: /content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2_LARGE/PLY/0000099.ply
Final depth min: 0.001 , max: 400.0
3D Recon .ply saved in: /content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2_LARGE/PLY/0000098.ply
Final depth min: 0.001 , max: 307.63965
3D Recon .ply saved in: /content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2_LARGE/PLY/0000107.ply
Final depth min: 0.001 , max: 332.1712
3D Recon .ply saved in: /content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2_LARGE/PLY/0000535.ply
Final depth min: 0.001 , max: 356.7084
3D Recon .ply saved in: /content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2_LARGE/PLY/0000554.ply
Final depth min: 0.001 , max: 235.46356
3D Recon .ply saved in: /content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2

Up scaling the pointcloud

In [ ]:
import open3d as o3d
import numpy as np

# Load the depth image (16-bit PNG, assumed to be in millimeters or meters)
depth_image = o3d.io.read_image("/content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/DA2_LARGE/Depth_Pred/0004502.png")
print(np.array(depth_image).min(), np.array(depth_image).max())

# Load the RGB image
color_image = o3d.io.read_image("/content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/masked_imgs/0004502.png")

# Ensure the depth image is in the correct format (convert if needed)
depth_image_np = np.asarray(depth_image)
if depth_image_np.dtype != np.float32:
    depth_image_np = depth_image_np.astype(np.float32) / 100.0  # Convert from mm to meters if needed
    depth_image = o3d.geometry.Image(depth_image_np)
    print('if inside:', np.array(depth_image).min(), np.array(depth_image).max())

# Create an RGBD image from the color and depth images
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_image, depth_image, convert_rgb_to_intensity=False, depth_scale=1.0, depth_trunc=3.0
)

# Define the camera intrinsic parameters
fx = 850.709  # focal length x
fy = 850.709  # focal length y
cx = 640  # principal point x
cy = 360  # principal point y
width = 1280
height = 720

# Create Open3D intrinsic object
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

# Create the point cloud from the RGBD image
point_cloud = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image, intrinsic
)

# Apply a scaling factor to the point cloud to make it bigger
scaling_factor = 10.0  # Adjust this factor to scale the point cloud
point_cloud.scale(scaling_factor, center=point_cloud.get_center())

# Flip the point cloud (optional, for better visualization in Open3D's coordinate system)
point_cloud.transform([[1, 0, 0, 0],
                       [0, -1, 0, 0],
                       [0, 0, -1, 0],
                       [0, 0, 0, 1]])

# Optionally, visualize the point cloud
# o3d.visualization.draw_geometries([point_cloud])

# Save the point cloud as a .ply file (with RGB colors)
o3d.io.write_point_cloud("/content/drive/MyDrive/Datasets_Weights/Endonasal/recon2/0004502v2_4.ply", point_cloud)


0 254
if inside: 0.0 2.54


True